# Top

Before `git add`:

* Edit, Clear Outputs of All Cells

## Imports

In [ ]:
from copy import deepcopy
from math import sin, asin, cos, acos, pi
import math
import numpy as np
import fullcontrol as fc

# Variables

In [ ]:
%whos

# Utility Functions

## Full Control

### GCode Banner

In [ ]:
def gcode_banner(text):
    steps = []
    steps.append(fc.GcodeComment(text='#' * 80))
    steps.append(fc.GcodeComment(text=f'# {text}'))
    steps.append(fc.GcodeComment(text='#' * 80))
    return steps

In [ ]:
# Mini Test
steps = []
steps.append(fc.Point(x=10, y=20, z=.2))
steps.extend(gcode_banner('hello'))

steps

### Square Spiral

In [ ]:
# Inspiration from:
# spiralXY(centre: Point, start_radius: float, end_radius: float, start_angle: float, n_turns: float, segments: int, cw: bool=False) -> list
# Spacing between each line is given by: size / n_turns / 2
def squareSpiral(center, size: float, n_turns: int, cw=False):
    # sub function
    def _spiral(outer_size, num_turns, cw):
        """
        Generate a square spiral centered on (0, 0) that reaches `outer_size` after `num_turns`.
        Returns a list of (x, y) coordinates.
        """
        if num_turns < 1:
            return [(0, 0)]

        num_segments = 4 * num_turns + 1

        step = outer_size / num_turns / 2
        x, y = 0, 0
        dx, dy = 1, 0  # initial direction: right
        coords = [(x, y)]

        for i in range(num_segments):
            f = (i // 2 + 1 * (i < num_segments - 1)) * step
            x += dx * f
            y += dy * f
            coords.append((x, y))
            if cw:
                dx, dy = dy, -dx  # rotate 90 degrees CW
            else:
                dx, dy = -dy, dx  # rotate 90 degrees CCW

        return coords

    steps = []
    for p in _spiral(size, n_turns, cw):
        steps.append(fc.Point(x=p[0] + center.x, y=p[1] + center.y, z=center.z))
    return steps

In [ ]:
# Mini test

print(
    fc.transform(
        squareSpiral(fc.Point(x=10, y=10, z=.2), 4, 2, True),
        'gcode',
        controls=fc.GcodeControls(
            printer_name='generic',  # or ender_3
            initialization_data={
                'extrusion_width': .4,
                'extrusion_height': .2
            }
        )
    )
)

### Hop To

In [ ]:
def hop_to(point, intermediate_points=None):
    """Similar to fc.travel_to(), but with retract."""
    steps = []

    steps.append(fc.PrinterCommand(id='retract'))  # Already performing a Z hop
    steps.append(fc.Extruder(on=False))
    if intermediate_points is not None:
        steps.extend(intermediate_points)
    steps.append(point)
    steps.append(fc.PrinterCommand(id='unretract'))
    steps.append(fc.Extruder(on=True))

    return steps

In [ ]:
# Mini test

print(
    fc.transform(
        [fc.Point(x=1, y=1, z=.2)] + hop_to(fc.Point(x=10, y=20)) + [fc.Point(x=20)] +
        [fc.Point(x=30, y=40, z=.2)] + hop_to(fc.Point(x=50, y=50), [fc.Point(y=100), fc.Point(x=110)]) + [fc.Point(x=20)],
        'gcode',
        controls=fc.GcodeControls(
            printer_name='generic',  # or ender_3
            initialization_data={
                'extrusion_width': .4,
                'extrusion_height': .2
            }
        )
    )
)

## General

### Circle Segments

In [ ]:
def circle_segments(radius, tolerance=0.05):
    """
    Calculate the number of line segments needed to approximate a circle.

    :param radius: Circle radius (mm)
    :param tolerance: Maximum allowed error (mm).  This is the distance from the center of the segment to the ideal circle tangent point.
    :return: Number of segments (int)
    """
    if radius <= 0:
        raise ValueError("Radius must be positive")
    if tolerance <= 0:
        raise ValueError("Tolerance must be positive")

    # exact formula
    n = pi / acos(1 - tolerance / radius)

    # round up to integer
    return max(10, math.ceil(n))

In [ ]:
# Mini test
[circle_segments(i) for i in range(1, 102, 10)]

### Paired Iterator

In [ ]:
# Ref: https://numpy.org/doc/stable/reference/generated/numpy.linspace.html
def paired_iter(x, y_ranges, endpoint=True):
    """Iterate over x and one or more y ranges in lockstep.

    x: array-like (from range, arange, linspace, ...)
    y_ranges: tuple(start, stop) or list of such tuples
    endpoint: If True, stop is the last sample [y_start, y_stop]. Otherwise, it is not included [y_start, y_stop[.
    """

    # Normalize to list of ranges
    if isinstance(y_ranges, tuple):
        y_ranges = [y_ranges]

    # Build all y arrays
    ys = [np.linspace(start, stop, len(x), endpoint) for start, stop in y_ranges]

    # Yield combined tuples
    for xi, *yi in zip(x, *ys):
        yield (xi, *yi)

In [ ]:
# Test 1
for x, y in paired_iter(range(3), (1, 2), True):
    print(f"{x:5.2f} {y:5.2f}")

In [ ]:
# Test 2
for x, y in paired_iter(np.linspace(1, 5, 5), (1, 2), False):
    print(f"{x:5.2f} {y:5.2f}")

In [ ]:
# Test 3
for x, y, z in paired_iter(np.linspace(0, 5, 6), [(1, 2), (10, 20)]):
    print(f"{x:8.2f} {y:8.2f} {z:8.2f}")

# Features

## Hotend Temperature

```
steps.append(fc.Hotend(temp=220, wait=True))
```

## Bed Temperature

```
steps.append(fc.Buildplate(temp=60, wait=True))
```

## Extruder on/off

```
# Travel
steps.extends(fc.travel_to(fc.Point(...)))

# Manuel
steps.append(fc.Extruder(on=False))
```

## Extrusion size

```
steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.4, height=0.2))

steps.append(fc.ExtrusionGeometry(area_model='circle', diameter=1))
```

## Print Speed

```
steps.append(fc.Printer(print_speed=750))
```

## Travel Speed

```
steps.append(fc.Printer(travel_speed=2000))
```

## Fan

```
steps.append(fc.Fan(speed_percent=50))
```

## Retraction

```
steps.append(fc.PrinterCommand(id='retract'))

steps.append(fc.PrinterCommand(id='unretract'))
```

## GCode

### Comments

```
steps.append(fc.GcodeComment(text='This is a comment'))
```

### Custom commands

```
steps.append(fc.Printer(new_command={'beep': 'M300 S440 P1000 ; beep'}))
```

### Manual GCode

```
steps.append(fc.ManualGcode(text='G4 P2000 ; pause for 2 seconds'))
```

# Designs

## Parameters

In [ ]:
# Printing Parameters

nozzle_temp = 220
bed_temp = 60
print_speed = 900  # TBD to confirm
travel_speed = 8000

EW = 0.5  # extrusion width
EH = 0.2  # extrusion height (and layer height)

## Simple

In [ ]:
steps = []

steps.append(fc.Extruder(on=False))
steps.append(fc.Point(x=10, y=10, z=.2))
steps.append(fc.Extruder(on=True))
steps.append(fc.Point(x=110))
steps.append(fc.Point(y=110))
steps.append(fc.Point(x=10, y=10))

steps

## Complex

### All in one

In [ ]:
def generate_steps():
    steps = []

    z = EH

    steps.append(fc.Point(x=2, y=2, z=z))
    steps.append(fc.Extruder(on=True))
    steps.append(fc.Point(y=200))
    steps.append(fc.Extruder(on=False))

    pt = fc.Point(x=20, y=20)
    steps.append(pt)

    steps.append(fc.Extruder(on=True))
    steps.extend(fc.rectangleXY(pt, 100, 10, cw=True))
    steps.append(fc.Extruder(on=False))

    pt = fc.Point(x=40, y=100)
    steps.append(pt)
    steps.append(fc.Extruder(on=True))
    steps.extend(fc.spiralXY(pt, 1, 10, 0, 10, 1000, True))
    steps.append(fc.Extruder(on=False))

    pt = fc.Point(x=20, y=40)
    steps.append(pt)
    steps.append(fc.Extruder(on=True))
    steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1, y=0, z=0), 10, 2, 30))
    steps.append(fc.Extruder(on=False))

    pt = fc.Point(x=100, y=100)
    steps.append(pt)
    steps.append(fc.Extruder(on=True))
    steps.extend(squareSpiral(pt, 80, 20, True))

    return steps

steps = generate_steps()

### Spiral

In [ ]:
def generate_steps():
    steps = []

    pt = fc.Point(x=100, y=100, z=EH)
    steps.extend(hop_to(pt))
    steps.extend(fc.spiralXY(pt, .1, 50, 0, 10, 2000, True))

    return steps

steps = generate_steps()

### Square Spiral

In [ ]:
def generate_steps():
    steps = []

    pt = fc.Point(x=100, y=100, z=EH)
    steps.extend(hop_to(pt))
    steps.extend(squareSpiral(pt, 40, 10, True))

    return steps

steps = generate_steps()

### Square Wave

In [ ]:
def generate_steps():
    steps = []

    pt = fc.Point(x=10, y=10, z=EH)
    steps.extend(hop_to(pt))
    steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1), 100, 10, 5, extra_end_line=True))

    return steps

steps = generate_steps()

### Polar Flower

In [ ]:
def polar_flower(theta, a, b, k):
    return k * cos(a * theta / b) + (1 - k)

In [ ]:
def generate_steps():
    steps = []

    steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.6, height=EH))

    # Manually need to tune this point!
    steps.extend(hop_to(fc.Point(x=80, y=50, z=EH)))

    for z in np.arange(EH, 5, EH):
        steps.extend(gcode_banner(f"Layer {z}"))
        if z > EH:
            steps.append(fc.Fan(speed_percent=100))
            steps.append(fc.Printer(print_speed=2 * print_speed))
        for i in np.linspace(0, 10 * pi, 1000):
            steps.append(
                fc.polar_to_point(
                    centre=fc.Point(x=50, y=50, z=z),
                    radius=30 * polar_flower(i, 13, 5, .45),
                    angle=i
                )
            )

    return steps

steps = generate_steps()

### Bowl

In [ ]:
def generate_steps():
    steps = []

    steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.5, height=EH))

    p = fc.Point(x=110, y=110, z=EH)
    r = 30

    # Layer 1
    steps.extend(hop_to(deepcopy(p)))
    # .4 spacing @ .5 mm width
    steps.extend(fc.spiralXY(
        deepcopy(p),
        .1,
        r,
        0,
        int(r / .4),
        int(r / .4) * circle_segments(r),
        True)
    )

    # Layer 2
    steps.append(fc.Fan(speed_percent=100))
    steps.append(fc.Printer(print_speed=2 * print_speed))
    p.z = 2 * EH
    steps.extend(hop_to(deepcopy(p)))
    steps.extend(fc.spiralXY(
        deepcopy(p),
        .1,
        r,
        0,
        int(r / .4),
        int(r / .4) * circle_segments(r),
        False)  # Reverse rotatoin for better adhesion
    )

    return steps

    a = .47
    k = .86
    for z, x1 in paired_iter(np.arange(2 * EH, 30, EH), (0, 2)):
        steps.extend(gcode_banner(f"Layer z={z:.3f}"))
        p.z = z
        r1 = r + 20 * sin(pi * x1) * (1 - math.tanh(k * (x - a))) / 2
        steps.extend(fc.circleXY(deepcopy(p), r, 0, circle_segments(r), True))

    print(len(steps))

    return steps

steps = generate_steps()

## Calibrations

### Bed Square Spiral

In [ ]:
def generate_steps():
    steps = []

    pt = fc.Point(x=110, y=110, z=EH)
    steps.extend(hop_to(pt))
    steps.extend(squareSpiral(pt, 200, 20, cw=True))

    return steps

steps = generate_steps()

### Line Width Tests

In [ ]:
def generate_steps():
    steps = []

    for i in range(15):
        w = (i + 1) / 10.0
        steps.append(fc.GcodeComment(text=f'Width = {w:.2f}'))
        steps.extend(hop_to(fc.Point(x=10 + 10 * i, y=10, z=EH)))
        steps.append(fc.ExtrusionGeometry(width=w))
        steps.append(fc.Point(y=110))  # Move 100 mm

    return steps

steps = generate_steps()

Conclusion:

* On bed
    * EH = .2
    * EW [.6, 1.0]

### Retraction Tests

Default:

```
>>> M207
<< M207 S5.00 W13.00 F2400.00 Z0.20

>>> M208
<< M208 S0.00 W0.00 F2400.00
```

From Orca gcode:
* Z hop is .4 (from .2 to .6)
* 3600 retract
* 2400 recover

In [ ]:
def generate_steps():
    steps = []

    steps.append(fc.Extruder(on=False))
    steps.append(fc.Point(x=5, y=5, z=EH))

    # Retraction: M207 F<feedrate> S<length>
    # Unretraction: M208 F<feedrate> S<length>.  Length is addition to M207 S

    # F: feedrate (units/min)  (3600 in gcode?)
    # S: length (mm)  0 to 10 ?

    steps.append(fc.ManualGcode(text=f"M207 Z0.4 ; retraction z hop"))
    steps.append(fc.ManualGcode(text=f"M207 F3600 ; retraction speed"))
    steps.append(fc.ManualGcode(text=f"M208 F2400 ; un-retraction speed"))

    for j in range(1):  # Feedrate
        #steps.append(fc.ManualGcode(text=f"M207 F{3600 + 10 * j:d} ; retraction speed"))
        #steps.append(fc.ManualGcode(text=f"M208 F{3600 + 10 * j:d} ; un-retraction speed"))
        for i in range(80):  # Length
            steps.append(fc.ManualGcode(text=f"M207 S{.1*(i+1):.2f} ; retraction length"))
            #steps.append(fc.ManualGcode(text=f"M208 S{.1 * i:.2f} ; un-retraction additional length"))
            steps.extend(hop_to(fc.Point(x=10 + 2 * i, y=10 + 15 * j)))
            steps.append(fc.Point(y=20 + 15 * j))

    return steps

steps = generate_steps()

### Fill Region Density

In Orca, 1st layer fill.

Gcode snippet:

```
;TYPE:Bottom surface
;WIDTH:0.500487
G1 F1200
G1 X10.25 Y10.25 E7.59817
G1 X10.25 Y10.708 E.01741
G1 X209.75 Y10.708 E7.59035
G1 X209.75 Y11.165 E.01741
```

Fill lines are spaced at 0.458

Print filled 1st layer results in lines sometimes not well fused together (when printing a simple filled square).

In [ ]:
# Delta Y
10.708 - 10.25

In [ ]:
# Delta X
209.75 - 10.25

In [ ]:
# Extruder:
7.59035 * pi * (1.75 / 2) ** 2

In [ ]:
_ / __

In [ ]:
_ / .2

In [ ]:
def generate_steps():
    steps = []

    y = 15
    dy = 18

    done = False

    for w in np.arange(.4, .8, .05):
        steps.extend(gcode_banner(f"Extrusion width = {w:.3f}"))
        steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=w, height=EH))
        x = 10
        dx = .3
        if not done:
            print(f"Starting dx: {dx}")

        steps.extend(hop_to(fc.Point(x=x, y=y, z=EH)))

        while x < 200 and dx < .8:
            steps.append(fc.GcodeComment(text=f"Delta X = {dx:.3f}"))
            steps.append(fc.Point(y=y + dy))  # up
            x += dx
            steps.append(fc.Point(x=x))  # left
            steps.append(fc.Point(y=y))  # down
            x += dx
            steps.append(fc.Point(x=x))  # left

            dx += .005    

            if not done:
                if int((1000 * dx)) % 100 == 0:
                    print(f"dx = {dx}")
                    steps.extend(hop_to(fc.Point(y=y-2)))
                    steps.append(fc.Point(y=y-8))
                    steps.extend(hop_to(fc.Point(y=y)))
        done = True
        y += 20
    print(f"Final dx: {dx}")
    return steps

steps = generate_steps()

#### Conclusion

Around .4 mm spacing gives good result with .5 mm LW.

### Print Speed

Could try with spiral, squareSpiral

With square wave:
* Size 5, 10 period
* All patterns had a good adhesion
* At higher speed, the cornering blob becomes more apparent.  Could use this pattern to tune the LA K factor?
* While watching it print, it seemed that the speed was capped somehow by the printer.

```
>>> M203
<< M203 X500.00 Y500.00 Z10.00 E60.00
```

60 mm/s max E

From OrcaSlicer (all in mm/s):

* First layer
    * layer: 15
    * infill: 20
* Other layers
    * Outer wall: 25
    * Inner wall: 40
    * Sparse infull: 50
    * Internal solid infill: 40
    * Top surface: 30
    * Gap infull: 30
    * Support: 150

#### Small Square Wave

This prints beautifully, but it seems that the small dimension limit the final speed.

Because of non-instant acceleration, the final speed cannot be reached.

In [ ]:
def generate_steps():
    steps = []

    size = 5  # dx, dy

    for i in range(25):
        s = 10 + 5 * i
        steps.append(fc.GcodeComment(text=f"Speed = {s} mm/s"))
        steps.append(fc.Printer(print_speed=60 * s))
        pt = fc.Point(x=10, y=10 + 7 * i, z=EH)
        steps.extend(hop_to(pt))
        steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1), size, size, 10, extra_end_line=True))

    print(s)

    return steps

steps = generate_steps()

#### Triangle Wave Pattern

80 mm/s seems the maximum without any artifacts.

In [ ]:
def generate_steps():
    steps = []

    size = 20  # dx, dy

    x0=10
    y=10

    steps.append(fc.ExtrusionGeometry(width=.6))  # wider lines
    # steps.append(fc.Point(x=5, y=2, z=EH))  # For plot only

    for j in range(10, 151, 5):
        steps.append(fc.Printer(print_speed=j * 60))
        steps.append(fc.GcodeComment(text=f"Printing at {j} mm/s"))
        x = x0
        steps.extend(hop_to(fc.Point(x=x, y=y, z=EH), [fc.Point(y=y + size + 5), fc.Point(x=x0)]))
        for i in range(5):
            x += size
            y += size
            steps.append(fc.Point(x=x, y=y))
            x += size
            y -= size
            steps.append(fc.Point(x=x, y=y))
        y += 5

    return steps

steps = generate_steps()

#### Long Lines

When looking at the printed result:

Formula is 10 mm/s + Delta_X mm/s / mm

Conclusion:
* At 90 mm/s: lines were uneven and not always adhering to bed
* At 105 mm/s: lines did not adhere at all to bed
* 85 mm/s would be the absolute max with good lines
* 80 mm/s would be the safe high value

In [ ]:
def generate_steps():
    steps = []

    y = 50
    dy = 100

    x = 10
    dx = 2.5

    steps.append(fc.ExtrusionGeometry(width=.6))  # wider lines
    steps.extend(hop_to(fc.Point(x=x, y=y, z=EH)))

    for s in range(10, 151, 5):
        steps.append(fc.Printer(print_speed=s * 60))
        steps.append(fc.GcodeComment(text=f"Printing at {s} mm/s"))

        steps.append(fc.Point(y=y + dy))  # up
        x += dx
        steps.append(fc.Point(x=x))  # left
        steps.append(fc.Point(y=y))  # down
        x += dx
        steps.append(fc.Point(x=x))  # left

    print(f"Final speed: {s}")

    return steps

steps = generate_steps()

#### Travel Only

In [ ]:
# Square wave
def generate_steps():
    steps = []
    steps.append(fc.Extruder(on=False))
    steps.extend(fc.squarewaveXY(fc.Point(x=10, y=10, z=2), fc.Vector(x=1), 20, 10, 15, extra_end_line=True))
    return steps

steps = generate_steps()

Conclusion:

Having too many points limits using directly the send serial...

In [ ]:
# Linear move
def generate_steps():
    steps = []
    steps.append(fc.Extruder(on=False))
    steps.append(fc.Point(x=10, y=10, z=2))
    for x in np.linspace(10, 200, 1000):
        steps.append(fc.Point(x=x))
    return steps

steps = generate_steps()

In [ ]:
# Spiral
def generate_steps():
    steps = []
    pt = fc.Point(x=100, y=100, z=2)
    steps.append(fc.Extruder(on=False))
    steps.append(pt)
    steps.extend(fc.spiralXY(pt, .1, 50, 0, 10, 2000, True))
    return steps

steps = generate_steps()

### Linear Advance K Factor

M900 Kx ; x (from 0 to 1)

#### Greater Than Pattern

Steps:
* Fill region to have a 1 st layer
* Perform angle shapes ( > sign)

In [ ]:
def generate_steps():
    steps = []

    size = 20  # dx, dy

    # Generate a filled 1st layer 
    pt = fc.Point(x=10, y=10, z=EH)
    #steps.extend(hop_to(pt))
    #steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1), 44, .5, 110))

    #steps.extend(hop_to(fc.Point(x=10, y=10, z=EH * 2)))  # next layer
    #steps.append(fc.Fan(speed_percent=100))  # Fan ON
    steps.append(fc.Printer(print_speed=100 * 60))  # 100 mm/s

    x = 10
    y = 10
    for j in np.arange(0, 1.0001, .05):
        steps.append(fc.ManualGcode(text=f"M900 K{j:.2f} ; linear advance k factor"))
        for i in range(3):
            #steps.extend(hop_to(fc.Point(x=x, y=y, z=EH * 2)))
            steps.extend(hop_to(fc.Point(x=x, y=y, z=EH)))
            x += size
            y += size
            steps.append(fc.Point(x=x, y=y))
            x -= size
            y += size
            steps.append(fc.Point(x=x, y=y))
            y -= 2 * size
            x += .5 / sin(pi / 4)

        x += 2

    return steps

steps = generate_steps()

#### Triangle Wave Pattern

After trying it on the SD card, many lines were not sticking properly, probably printing to fast.  100 mm/s on 1st layer is too high.

In [ ]:
def generate_steps():
    steps = []

    steps.append(fc.Printer(print_speed=80 * 60))  # x60 to be in /min
    steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.6, height=EH))

    size = 20  # dx, dy

    x0=10
    y=10

    for j in np.arange(0, 1.0001, .05):
        steps.append(fc.ManualGcode(text=f"M900 K{j:.2f} ; linear advance k factor")) 
        x = x0
        steps.extend(hop_to(fc.Point(x=x, y=y, z=EH), [fc.Point(y=y + size + 5), fc.Point(x=x0)]))
        for i in range(5):
            x += size
            y += size
            steps.append(fc.Point(x=x, y=y))
            x += size
            y -= size
            steps.append(fc.Point(x=x, y=y))
        y += 5

    return steps

steps = generate_steps()

With 60 mm/s, .6 width, the 11th pattern from bottom seemed the best.

This gives a K of:

In [ ]:
(11-1) * .05

#### Line speed change

From OrcaSlicer test:

* 600 mm/min slow
* 6000 mm/min fast
* 20 mm 1st slow path
* 40 mm fast path
* 20 mm 2nd slow path

```
G1 X80 ...
G1 F6000
G1 X120 Y20.75 E1.85285
```

In [ ]:
# Delta X
120 - 80

In [ ]:
# Extruder:
1.85285 * pi * (1.75 / 2) ** 2

In [ ]:
_ / __

In [ ]:
# Line width:
_ / .2

In [ ]:
def generate_steps():
    steps = []

    steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.6, height=EH))

    # Parameters
    size = 40
    x0 = 10
    y = 20

    marker_y = 5
    marker_size = 10

    # in mm/min
    fast_speed = 80 * 60
    slow_speed = fast_speed / 10.0

    steps.append(fc.Printer(print_speed=slow_speed))
    steps.extend(hop_to(fc.Point(x=x0 + size, y=marker_y, z=EH)))
    steps.append(fc.Point(y=marker_y + marker_size))
    steps.extend(hop_to(fc.Point(x=x0 + 3 * size, y=marker_y, z=EH)))
    steps.append(fc.Point(y=marker_y + marker_size))

    for j in np.arange(0, 1.0001, .02):
        steps.append(fc.ManualGcode(text=f"M900 K{j:.2f} ; linear advance k factor")) 
        x = x0
        steps.extend(hop_to(fc.Point(x=x, y=y, z=EH)))
        # Slow
        steps.append(fc.Printer(print_speed=slow_speed))
        x += size
        steps.append(fc.Point(x=x))
        # Fast
        steps.append(fc.Printer(print_speed=fast_speed))
        x += 2 * size
        steps.append(fc.Point(x=x))
        # Slow
        steps.append(fc.Printer(print_speed=slow_speed))
        x += size
        steps.append(fc.Point(x=x))
        # Move up
        y += 2.5

    return steps

steps = generate_steps()

K = 0.02 * Delta_Y / 2.5

Conclusion:
* From K = .48, the slow to fast transition was uniform, but the fast to slow transition was still not good.
* At K = 0.58, both transisions were uniform.

In [ ]:
60 / 2.5 * .02

In [ ]:
72.5 / 2.5 * .02

# Visualization

In [ ]:
fc.transform(
    steps,
    'plot',
    fc.PlotControls(
        #neat_for_publishing=True,
        style='line',
        #style='tube',
        zoom=.9,
        initialization_data={'extrusion_width': EW, 'extrusion_height': EH}
    )
)

# GCODE Generation

## Basic

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    controls=fc.GcodeControls(
        printer_name='generic',  # or ender_3
        initialization_data={
            'extrusion_width': EW,
            'extrusion_height': EH
        },
        save_as='fullcontrol',
        include_date=False
    )
)

for l in gcode.splitlines()[:40]:
    print(l)

## Ender 3 (built-in)

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    fc.GcodeControls(
        printer_name='ender_3',
        initialization_data={
            'primer': 'travel',  # front_lines_then_y
            'travel_speed': travel_speed,
            'print_speed': print_speed,
            'nozzle_temp': nozzle_temp,
            'bed_temp': bed_temp,
            'fan_percent': 0,
            'extrusion_width': EW,
            'extrusion_height': EH},
        save_as='fullcontrol',
        include_date=False
    )
)

for l in gcode.splitlines()[:40]:
    print(l)

## GCode Generate Function

In [ ]:
# Custom for Ender 3 + BLTouch
def to_gcode(steps, filename='fullcontrol'):
    init = []
    init.extend(gcode_banner('STARTING PROCEDURE'))
    init.append(fc.Printer(new_command={'beep': 'M300 S440 P1000 ; beep'}))
    init.append(fc.PrinterCommand(id='beep'))
    init.append(fc.ManualGcode(text='G90 ; use absolute coordinates'))
    init.append(fc.ManualGcode(text="M207 Z0.4 ; retraction z hop"))
    init.append(fc.ManualGcode(text="M207 F3600 ; retraction speed"))
    init.append(fc.ManualGcode(text="M207 S5 ; retraction length"))
    init.append(fc.ManualGcode(text="M208 F2400 ; recover speed"))
    init.append(fc.ManualGcode(text="M208 S0 ; additional recover length"))
    init.append(fc.ManualGcode(text="M900 K0.58 ; linear advance K factor"))

    init.append(fc.Extruder(relative_gcode=True))
    init.append(fc.ManualGcode(text="G92 E0 ; reset extruder position to 0"))
    init.append(fc.ExtrusionGeometry(area_model='rectangle', width=EW, height=EH))

    init.append(fc.PrinterCommand(id='home'))
    init.append(fc.ManualGcode(text='G29 L0 ; load mesh from slot 0'))
    init.append(fc.ManualGcode(text='G29 F10 ; set fade height for correction at 10 mm'))
    init.append(fc.ManualGcode(text='G29 A ; activate UBL'))

    init.append(fc.Extruder(on=False))
    init.append(fc.Printer(travel_speed=travel_speed, print_speed=print_speed))
    init.append(fc.Fan(speed_percent=0))  # To start off for first layer
    init.append(fc.Hotend(temp=nozzle_temp, wait=False))
    init.append(fc.Buildplate(temp=bed_temp, wait=False))
    init.append(fc.Point(x=2, y=2, z=50))

    init.append(fc.Hotend(temp=nozzle_temp, wait=True))
    init.append(fc.Buildplate(temp=bed_temp, wait=True))
    init.append(fc.PrinterCommand(id='beep'))

    init.extend(gcode_banner('PRIMER LINES'))
    init.append(fc.Point(z=EH))
    init.append(fc.Extruder(on=True))
    init.append(fc.Point(y=200))
    init.append(fc.Point(x=2.4))
    init.append(fc.Point(y=2))
    init.append(fc.Extruder(on=False))

    init.extend(gcode_banner('PRINTING STEPS'))

    ending = []
    ending.extend(gcode_banner('START OF ENDING PROCEDURE'))
    ending.append(fc.PrinterCommand(id='retract'))
    ending.append(fc.Extruder(on=False))
    ending.append(fc.ManualGcode(text="G91 ; Set all axes to relative"))
    ending.append(fc.ManualGcode(text=f"G0 Z50 ; Move up above design"))
    ending.append(fc.ManualGcode(text="G90 ; Set all axes to absolute"))
    ending.append(fc.Point(x=2, y=220))
    ending.append(fc.ManualGcode(text='G4 ; wait motion to complete'))
    ending.append(fc.ManualGcode(text='M107 ; Fan Off'))
    ending.append(fc.Hotend(temp=0, wait=False))
    ending.append(fc.Buildplate(temp=0, wait=False))
    ending.append(fc.ManualGcode(text='M84 X Y E ; disable motors (except Z)'))
    ending.append(fc.PrinterCommand(id='beep'))
    ending.extend(gcode_banner('END OF ENDING PROCEDURE'))

    gcode = fc.transform(
        init + steps + ending,
        'gcode',
        fc.GcodeControls(
            printer_name='custom',
            initialization_data={
                #'primer': 'travel',
                #'print_speed': print_speed,
                #'travel_speed': travel_speed,
                #'nozzle_temp': nozzle_temp,
                #'bed_temp': bed_temp,
                #'fan_percent': 0,
                'extrusion_width': EW,
                'extrusion_height': EH,
            },
            save_as=filename,
            include_date=False
        )
    )
    return gcode

### Output

In [ ]:
#print(to_gcode(steps))
to_gcode(steps) ;